In [1]:
# Plotting notebook for the project
# Imports
import argparse
import os
import sys
import glob
import re

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
from datetime import datetime
import scipy.stats as stats

# import cdo
from cdo import *
cdo = Cdo()

# Local imports
sys.path.append('/home/users/benhutch/skill-maps-historical')
import dictionaries as dic
import functions as fnc

In [2]:
# Set up the parameters again
# for the years 2-9 psl ULG (JJA)
var = "psl"
region = "global"
region_grid = dic.gridspec_global
forecast_range = "2-9"
season = "DJFM" # weird season name for model
observations_path = dic.obs
obs_var_name = "psl"
model_dict = dic.model_dictionary_psl_historical_badc
start_year = 1960
end_year = 2014

In [3]:
# # Process the observations first of all
# obs = fnc.process_observations(var, region, region_grid,
#                                forecast_range, season, observations_path,
#                                obs_var_name)

In [4]:
# # First we want to merge the time axis and regrid the model (historical) data
# # using the function call_mergetime_regrid
# # this does not return anything, but saves the regridded data to a netcdf file
# fnc.call_mergetime_regrid(model_dict, var, region)

In [5]:
# Now we want to load the historical data
# as a dictionary of xarray datasets for each model
# using the load_historical_data function
historical_data = fnc.load_historical_data(model_dict, var, region)

# historical_data

processing model:  BCC-CSM2-MR
type of var <class 'str'>
type of model <class 'str'>
type of region <class 'str'>
regrid_files: /gws/nopw/j04/canari/users/benhutch/historical/psl/BCC-CSM2-MR/regrid/psl_Amon_BCC-CSM2-MR_historical_r*i?p?f?_*global_regrid.nc
number of files for model  BCC-CSM2-MR :  3
processing model:  MPI-ESM1-2-HR
type of var <class 'str'>
type of model <class 'str'>
type of region <class 'str'>
regrid_files: /gws/nopw/j04/canari/users/benhutch/historical/psl/MPI-ESM1-2-HR/regrid/psl_Amon_MPI-ESM1-2-HR_historical_r*i?p?f?_*global_regrid.nc
number of files for model  MPI-ESM1-2-HR :  1
processing model:  CanESM5
type of var <class 'str'>
type of model <class 'str'>
type of region <class 'str'>
regrid_files: /gws/nopw/j04/canari/users/benhutch/historical/psl/CanESM5/regrid/psl_Amon_CanESM5_historical_r*i?p?f?_*global_regrid.nc
number of files for model  CanESM5 :  35
processing model:  CMCC-CM2-SR5
type of var <class 'str'>
type of model <class 'str'>
type of region <cl

In [6]:
historical_data['NorCPM1'][0].psl.values

array([[[ 99907.49 ,  99907.49 ,  99907.49 , ...,  99907.49 ,
          99907.49 ,  99907.49 ],
        [ 99704.44 ,  99698.53 ,  99687.21 , ...,  99703.35 ,
          99702.11 ,  99706.84 ],
        [ 99341.79 ,  99330.51 ,  99316.67 , ...,  99388.89 ,
          99369.04 ,  99355.89 ],
        ...,
        [ 99749.84 ,  99769.48 ,  99788.29 , ...,  99692.47 ,
          99711.65 ,  99730.5  ],
        [ 99728.06 ,  99741.79 ,  99755.38 , ...,  99689.414,
          99702.01 ,  99714.76 ],
        [ 99731.17 ,  99738.805,  99746.59 , ...,  99709.98 ,
          99716.3  ,  99723.42 ]],

       [[100587.39 , 100587.39 , 100587.39 , ..., 100587.39 ,
         100587.39 , 100587.39 ],
        [100250.94 , 100252.36 , 100250.42 , ..., 100248.47 ,
         100248.2  , 100251.02 ],
        [ 99842.51 ,  99847.336,  99851.84 , ...,  99848.414,
          99843.57 ,  99843.27 ],
        ...,
        [ 99868.92 ,  99855.97 ,  99846.04 , ...,  99930.055,
          99905.77 ,  99885.22 ],
        [100

In [7]:
# now we want to call the function to process the historical data
# this will:
# 1. constrain the data to the provided year range and season
# 2. calculate the climatology and remove this to create anomalies
# 3. calculate the annual mean anomalies from the monthly mean anomalies
# 4. calculate the running mean of these annual mean anomalies
# then add the processed data back into the dictionary
processed_historical_data = fnc.process_historical_data(historical_data, season, forecast_range, start_year, end_year)

processing model:  BCC-CSM2-MR
processing member:  0
type of data in constrain historical data season <class 'xarray.core.dataset.Dataset'>
data:  <xarray.Dataset>
Dimensions:    (time: 1980, bnds: 2, lon: 144, lat: 72)
Coordinates:
  * time       (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * lon        (lon) float64 -180.0 -177.5 -175.0 -172.5 ... 172.5 175.0 177.5
  * lat        (lat) float64 -90.0 -87.5 -85.0 -82.5 ... 80.0 82.5 85.0 87.5
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object dask.array<chunksize=(50, 2), meta=np.ndarray>
    psl        (time, lat, lon) float32 dask.array<chunksize=(50, 72, 144), meta=np.ndarray>
Attributes: (12/51)
    CDI:                    Climate Data Interface version 2.0.4 (https://mpi...
    Conventions:            CF-1.7 CMIP-6.2
    source:                 BCC-CSM 2 MR (2017):   aerosol: none  atmos: BCC_...
    institution:            Beijing Climate Center, Beijing 100081, China
    activi

In [8]:
# Now we want to process the historical data
# in preperation for calculating the spatial correlations
# this function constrains the years to only those available in all members
# and then calculates the equally weighted ensemble mean of all members
# using the function process_historical_data_spacial_correlations
ensemble_mean = fnc.process_historical_data_spatial_correlations(processed_historical_data)

TypeError: 'NoneType' object is not iterable